In [7]:
from pathlib import Path
import pandas as pd

In [10]:
path = Path("cleaned_post_code_registration.csv")

In [11]:
read0 = pd.read_csv(path)

In [12]:
read0.head()

,Postcode,State,Fuel Type,Registrations as at 31 January 2021,Registrations as at 31 January 2022,Registrations as at 31 January 2023
0,800,NT,Internal Combustion Engine,6780,7318,7089
1,800,NT,Battery Electric Vehicle,3,3,12
2,800,NT,Hydrogen Cell Vehicle,0,0,0
3,800,NT,Hybrid,131,411,402
4,810,NT,Internal Combustion Engine,21870,22092,21890


In [13]:
read0.count()

Postcode                               10548
State                                  10548
Fuel Type                              10548
Registrations as at 31 January 2021    10548
Registrations as at 31 January 2022    10548
Registrations as at 31 January 2023    10548
dtype: int64

In [14]:
read0.dtypes


Postcode                                int64
State                                  object
Fuel Type                              object
Registrations as at 31 January 2021     int64
Registrations as at 31 January 2022     int64
Registrations as at 31 January 2023     int64
dtype: object

In [15]:
read0["Fuel Type"].value_counts()

Internal Combustion Engine    2637
Battery Electric Vehicle      2637
Hydrogen Cell Vehicle         2637
Hybrid                        2637
Name: Fuel Type, dtype: int64

In [16]:
read0["State"].value_counts()

VIC    2776
NSW    2440
QLD    1732
WA     1536
SA     1364
TAS     460
NT      136
ACT     104
Name: State, dtype: int64

In [17]:
read0["Fuel Type"] = read0["Fuel Type"].replace({"BEV":"Battery Electric Vehicle",
                       'ICE':"Internal Combustion Engine",
                       'HFCEV':'Hydrogen Cell Vehicle',
                       'Hybrid/PHEV':'Hybrid'})

In [18]:
read0.head()

,Postcode,State,Fuel Type,Registrations as at 31 January 2021,Registrations as at 31 January 2022,Registrations as at 31 January 2023
0,800,NT,Internal Combustion Engine,6780,7318,7089
1,800,NT,Battery Electric Vehicle,3,3,12
2,800,NT,Hydrogen Cell Vehicle,0,0,0
3,800,NT,Hybrid,131,411,402
4,810,NT,Internal Combustion Engine,21870,22092,21890


In [19]:
read1= read0.groupby(["Postcode","Fuel Type"])
read1

In [20]:
read0.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
10543    False
10544    False
10545    False
10546    False
10547    False
Length: 10548, dtype: bool